# Gene mapping

![](./images/Module1/Gene_ID_Conversion.png)
*(Source: david.ncifcrf.gov)*

## Learning Objectives:
1. Installing gene ID database
2. Performing gene ID mapping using R command

### Downloading data using R command line

If the users have programming background, it is more beneficial to automate the downloading task using R's command line. Getting data from GEO is really quite easy using <a href="https://bioconductor.org/packages/release/bioc/html/GEOquery.html">GEOquery</a> R package available in <a href="https://bioconductor.org/">Bioconductor</a>. Before starting, user need to install GEOquery package using the following command.

We can use the `getGEO` function from the `GEOquery` package to download GEO dataset. First, users have to specify the accession ID of the dataset. For demonstration, we will use the same dataset `GSE48350`.

Some datasets on GEO may be derived from different microarray platforms. Therefore the object `gse` is a list of different datasets.
You can find out how many were used by checking the length of the `gse` object.

The result shows that we have only one dataset that belongs to microarray platform mentioned GEO dataset page. Then, we can access the first object in the list and explore the samples and genes metadata using the command below:

## Data normalization
In this section, we will inspect the data to discover what scale the data are presented in. The methods we will use later in this training series assume the data are on a `log2` scale; typically in the range of `0` to `16`.

The `exprs` function can retrieve the expression values as a data frame; with one column per-sample and one row per-gene. To view the expression value of the first 5 samples, we can use the following command:
